In [1]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment

from pathlib import Path

import copy 
import pandas as pd
import datetime

from scipy.signal import find_peaks

import os

# get dictools stuff 
import sys
sys.path.append("c:/work/hrdic-tools/")
import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

Loading ITKPyBase... Loading ITKPyBase... Loading ITKCommon... Loading ITKCommon... Loading ITKStatistics... Loading ITKStatistics... Loading ITKImageFilterBase... Loading ITKImageFilterBase... Loading ITKTransform... Loading ITKTransform... Loading ITKImageFrequency... Loading ITKImageFrequency... Loading ITKIOImageBase... Loading ITKIOBMP... Loading ITKIOBMP... Loading ITKIOBioRad... Loading ITKIOBioRad... Loading ITKIOBruker... Loading ITKIOBruker... Loading ITKIOGDCM... Loading ITKIOGDCM... Loading ITKIOIPL... Loading ITKIOIPL... Loading ITKIOGE... Loading ITKIOGE... Loading ITKIOGIPL... Loading ITKIOGIPL... Loading ITKIOHDF5... Loading ITKIOHDF5... Loading ITKIOJPEG... Loading ITKIOJPEG... Loading ITKIOJPEG2000... Loading ITKIOJPEG2000... Loading ITKIOTIFF... Loading ITKIOTIFF... Loading ITKIOLSM... Loading ITKIOLSM... Loading ITKIOMINC... Loading ITKIOMINC... Loading ITKIOMRC... Loading ITKIOMRC... Loading ITKIOMeta... Loading ITKIOMeta... Loading ITKIONIFTI... Loading ITKIONIFTI

<itkTemplate itk::TileConfiguration>
Options:
  [2,]
  [3,]
  [4,]


In [2]:
def newtec_rigdata_read(rig_file_name,gauge_length,gauge_width,gauge_thickness):
    # reads in the two key datafiles from Softstrain
    # 1. csv file containing the rig data
    # 2. log file containing imaging and step timings 

    # This function combines everything into one giant dataframe. Unfortunately the timestamps are different for the log and the csv so we have to do some "filling"

    # Read the csv file 
    csv_file = rig_file_name + '.csv'
    df_csv = pd.read_csv(csv_file)

    # change to work with timestamps rather than "Time"
    df_csv['Time stamp'] = pd.to_datetime(df_csv['Time'])

    # get rid of these as metadata may be wrong meaning these are wrong
    df_csv.drop(columns = ['Strain (%)','Stress (MPa)','Corrected Elongation (µm)','Corrected Strain (%)','Time'],inplace=True)


    # Read log file 
    # read log file to get everything 
    log_file = rig_file_name + '_log.txt'
    df_log = pd.read_csv(log_file,names=['Raw'])

    # separate out lines into timestamps and content 
    df_log[['Time stamp','Text']] = df_log['Raw'].str.split(pat=r"PROJECT|SEM MANAGER|TRACTION",expand=True,regex=True)
    df_log['Time stamp'] = df_log['Time stamp'].str.replace('[INFO] ','')

    df_log['Time stamp'] = pd.to_datetime(df_log['Time stamp'])

    # separate out SEM MANAGER lines 
    df_log['Type'] = None
    df_log.loc[pd.notnull(df_log['Raw'].str.split(pat=r"SEM MANAGER",expand=True,regex=True)[1]),'Type'] = 'SEM MANAGER'

    # separate out PROJECT lines
    df_log.loc[pd.notnull(df_log['Raw'].str.split(pat=r"PROJECT",expand=True,regex=True)[1]),'Type'] = 'PROJECT'

    # separate out PROJECT lines
    df_log.loc[pd.notnull(df_log['Raw'].str.split(pat=r"TRACTION",expand=True,regex=True)[1]),'Type'] = 'TRACTION'

    
    # Join the two dataframes together
    # join everything together
    df_mega = pd.concat([df_csv,df_log])

    # sort so timestamps increase monotonically
    df_mega = df_mega.sort_values('Time stamp')

    # reindex to destroy old indices from previous dataframes
    df_mega = df_mega.reset_index(drop=True)

    # tidy up some unnecessary columns
    df_mega = df_mega.drop(columns = ['Raw',
                                    'Elapsed Time (s)',
                                    'LVDT Position (µm)'])



    # fill in blanks for timestamps that don't match imaging timestamps 
    # backfill first
    df_mega['Position (µm)']            = df_mega['Position (µm)'].bfill()
    df_mega['Elongation (µm)']          = df_mega['Elongation (µm)'].bfill()
    df_mega['Force (N)']                = df_mega['Force (N)'].bfill()
    df_mega['Step']                     = df_mega['Step'].bfill()
    df_mega['Sample Temperature (°C)']  = df_mega['Sample Temperature (°C)'].bfill()

    

    # then foward fill to deal with the very end
    df_mega['Position (µm)']            = df_mega['Position (µm)'].ffill()
    df_mega['Elongation (µm)']          = df_mega['Elongation (µm)'].ffill()
    df_mega['Force (N)']                = df_mega['Force (N)'].ffill()
    df_mega['Step']                     = df_mega['Step'].ffill()
    df_mega['Sample Temperature (°C)']  = df_mega['Sample Temperature (°C)'].ffill()


    # calculate stress and strain measures 
    gauge_csa = gauge_width*gauge_thickness*1e-6 # m^2
    df_mega['Eng Stress / MPa']  = df_mega['Force (N)']*1e-6/gauge_csa
    df_mega['Eng Strain / - ']   = (df_mega['Elongation (µm)']-df_mega['Elongation (µm)'].iloc[0])*1e-3 / gauge_length

    df_mega['True Stress / MPa'] = df_mega['Eng Stress / MPa']*(1 + df_mega['Eng Strain / - '])
    df_mega['True Strain / - ']  = np.log(1+df_mega['Eng Strain / - '])

    # add one to steps as (for some unknown reason), Softstrain starts at -1
    df_mega['Step'] = df_mega['Step'] + 1

    # make a new elapsed time column
    t_delta = df_mega['Time stamp']- df_mega['Time stamp'].iloc[0]

    # convert to seconds. Pandas by default goes to nanoseconds so convert to seconds
    t_delta = 1e-9*pd.to_numeric(t_delta)
    df_mega['Elapsed Time (s)'] = t_delta


    # find imaging steps
    df_mega['Imaging step'] = False 
    df_mega['Imaging start'] = False

    for i in range(0,np.asarray(df_mega['Step'].unique(),dtype=int)[-1]+1):
        this_step = df_mega[df_mega['Step'] == i] 

        # word that triggers this is "Acquisition" - find first occurence
        try:
            # checks if step contains an "Acquisition"
            idx = this_step['Text'][this_step['Text'].str.contains('Acquisition',na=False)].index[0]

            # sets imaging step boolean to true if there is an acquisition
            df_mega['Imaging step'] = df_mega['Imaging step'].mask(df_mega['Step'] == i,True)

            # sets imaging start boolean to true for time stamp of imaging start
            df_mega.at[idx,'Imaging start'] = True

        except:
            # not an imaging step so we can ignore 
            pass






    # plot a figure to show the test profiles
    fig = plt.figure()

    fig.set_size_inches(12, 7)

    # set up grid 
    gs0 = gridspec.GridSpec(nrows=1,ncols=2,figure=fig,width_ratios=[1,3])
    gs00 =gridspec.GridSpecFromSubplotSpec(nrows=3,ncols=1,subplot_spec=gs0[0])
    gs01 =gridspec.GridSpecFromSubplotSpec(nrows=1,ncols=1,subplot_spec=gs0[1:])

    ax_small = gs00.subplots(sharex=True)

    ax_big = fig.add_subplot(gs01[0])

    for i in range(0,np.asarray(df_mega['Step'].unique(),dtype=int)[-1]+1):
        this_step = df_mega[df_mega['Step'] == i]

        #plot time as elapsed from start 

        # force - time
        ax_small[0].plot(this_step['Elapsed Time (s)']/3600,this_step['Force (N)'])

        # elongation - time
        ax_small[1].plot(this_step['Elapsed Time (s)']/3600,this_step['Elongation (µm)'])

        # Temperature - time
        ax_small[2].plot(this_step['Elapsed Time (s)']/3600,this_step['Sample Temperature (°C)'])

        # Engineering stress - engineering strain
        ax_big.plot(this_step['Eng Strain / - '],this_step['Eng Stress / MPa'])

        # plot markers for when imaging starts 
        im_start = this_step[this_step['Imaging start'] == True]
        ax_big.plot(im_start['Eng Strain / - '],im_start['Eng Stress / MPa'],'ko')

        # add annotation for step number
        try:
            x0 = im_start['Eng Strain / - '].to_numpy()[0]
            y0 = im_start['Eng Stress / MPa'].to_numpy()[0]
            dx = 0.00075
            dy = -10
            
            ax_big.annotate(str(i),xy = (x0,y0), xytext = (x0+dx,y0+dy),arrowprops=dict(arrowstyle='-',linewidth=1,shrinkB=5))
        except: 
            pass

    # label it all up
    # fig.suptitle('Test profile')

    # time dependent graphs
    ax_small[-1].set_xlabel('Time / hours')

    ax_small[0].set_ylabel('Force / N')
    ax_small[1].set_ylabel('Elongation (µm)')
    ax_small[2].set_ylabel('Sample Temperature (°C)')

    # flow curve
    ax_big.set_xlabel('Engineering strain / -')
    ax_big.set_ylabel('Engineering Stress / MPa')

    # switch grids on 
    for ax in ax_small: ax.grid()
    ax_big.grid()


    plt.tight_layout()


    # plot figure to show imaging times as a function of force vs time 
    fig,ax = plt.subplots()

    fig.set_size_inches(12, 7)
    for i in range(0,np.asarray(df_mega['Step'].unique(),dtype=int)[-1]+1):
        this_step = df_mega[df_mega['Step'] == i]

        #plot time as elapsed from start 

        # force - time
        ax.plot(this_step['Elapsed Time (s)']/3600,this_step['Force (N)'])

        # plot markers for when imaging starts 
        im_start = this_step[this_step['Imaging start'] == True]
        ax.plot(im_start['Elapsed Time (s)']/3600,im_start['Force (N)'],'ko')

        # add annotation for step number
        try:
            x0 = im_start['Elapsed Time (s)'].to_numpy()[0]/3600
            y0 = im_start['Force (N)'].to_numpy()[0]
            dx = 1
            dy = -20
            
            ax.annotate(str(i),xy = (x0,y0), xytext = (x0+dx,y0+dy),arrowprops=dict(arrowstyle='-',linewidth=1,shrinkB=10))#width=1,facecolor='black',headwidth=1,headlength=5,shrink=0.2))
        except: 
            pass

            
    ax.grid()
    ax.set_xlabel('Time / hours')
    ax.set_ylabel('Force / N')
    
    plt.tight_layout()

    return df_mega

In [3]:
gauge_length = 12.0      # mm
gauge_width = 1.5        # mm
gauge_thickness = 1.006  # mm
rig_data_file = './ofhc_cu_5pc'


df = newtec_rigdata_read('./ofhc_cu_5pc',gauge_length,gauge_width,gauge_thickness)